In [ ]:
# Sorting algorithm
# Sort odd decreasing then even increasing


In [ ]:
def house_sort(lst):
    '''
    Sorts input list by all odd numbers descending then all even numbers ascending
    '''
    
    

In [137]:
lst =  [-1, 2.2, 9, 5, 4, 7, 10]

In [51]:
# Algorithm 1
first_list = []
second_list = []
for x in lst:
    if round(x)%2 ==1:
        first_list.append(x)
    else:
        second_list.append(x)

first_list.sort(reverse = True)
second_list.sort(reverse = False)
first_list + second_list

# Big(O) = N+ 1/2NLOGN + 1/2NLOGN - check this

[9, 7, 5, -1, 2.2, 4, 10]

In [143]:
# list comprehension
a = [x for x in lst if x%2==1]
a.sort(reverse = True)
b = [x for x in lst if x%2==0]
b.sort(reverse = True)
a+b

[9, 7, 5, -1, 10, 4]

In [148]:
# in one line
sorted([x for x in lst if x%2==1], reverse = True) + sorted([x for x in lst if x%2==0])

[9, 7, 5, -1, 4, 10]

In [60]:
lst =  [-1, 2.2, 9, 5, 4, 7, 10]
# unnecesarily complex as pop changes list so index no longer references right element
first_list = []
for i,x in enumerate(lst):
    if x%2 ==1:
        first_list.append(lst.pop(i))
    else:
        second_list.append(lst.pop(i))
        
# etc

[-1, 9, 7]

In [155]:
# filter
lst =  [-1, 2.2, 9, 5, 4, 7, 10]
# filter applies function to every iterable - like a matrix then is it more efficient? what if doing it twice? is there a multiple output option (so only apply filter once)
# include something like a pop in the lambda function?
sorted(list(filter(lambda x: x%2 == 1,lst)), reverse = True) + sorted(list(filter(lambda x: x%2 == 0,lst)))

[9, 7, 5, -1, 4, 10]

In [62]:
# is there a way of applying a matrix to list so that evens are filtered out?

2.2

In [ ]:
# modifying a sort algorithm for my purpose?
# e.g. Timsort, but if even - multiply by largest no. so that left till end

#### Exception handling

In [ ]:
# Handling possible cases
# A float? - no reason but should round to nearest number
# A string? - ignore and raise a note?

# 7a 7b, want it to decrease if odd and increase if even
# -ve numbers? - treat them as normal (low)
# 0? - treat as even as in instructions

# Handling inputs
# Want it as an array
# if not array then convert it - exceptions around converting to array?
    if any iterable then ok?
    list, set, tuple, (convert)
    dictionary (use values and convert?)
    dataframe - should not occur
    series? - convert
    
    string? - assume csv or error?
    int, float bool input? - error

In [ ]:
# Handling input itself

In [122]:
lst = [1,2]
lst = {1,2}
lst = (1,2)
lst = {1:2,2:1}
arr = np.array(list(lst.values()))


array([2, 1])

In [134]:
# np.array(split('str, sf'),',')
x = 'str, sy'
x.split(', ')

['str', 'sy']

In [ ]:
# Handling elements
# float round(x) or treat like 7a? - always increasing
# duplicates? - remove
# -v - nothing
# is a string - ignore (error handling)
# is a list - unpack
